In [1]:
!pip install -q accelerate peft transformers trl datasets isbnlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import GPT2Tokenizer, GPT2Model
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    pipeline,
    logging,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

model_name = "gpt2-large"  # This is the base GPT-2 model with 124M parameters
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Use the end-of-sequence token as padding token
model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')

# Set pad token for the model as well
model.config.pad_token_id = tokenizer.eos_token_id


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
# 3. Prepare the dataset
dataset = load_dataset("P1ayer-1/isbndb-full-database")
dataset = dataset.shuffle(seed=42)
dataset = dataset['train'].select(range(20000))
df = dataset.to_pandas()

README.md:   0%|          | 0.00/671 [00:00<?, ?B/s]

(…)-00000-of-00011-8c76aaae89c32750.parquet:   0%|          | 0.00/230M [00:00<?, ?B/s]

(…)-00001-of-00011-0570d9f1f07ee5ea.parquet:   0%|          | 0.00/222M [00:00<?, ?B/s]

(…)-00002-of-00011-43bde59cf032f28e.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00003-of-00011-cc6f9f6e8311bb1d.parquet:   0%|          | 0.00/300M [00:00<?, ?B/s]

...

(…)-00007-of-00011-ea8e09c51301bce7.parquet:   0%|          | 0.00/235M [00:00<?, ?B/s]

(…)-00008-of-00011-bff7d5ac18f4238c.parquet:   0%|          | 0.00/297M [00:00<?, ?B/s]

(…)-00009-of-00011-84ef5d0ca46f90be.parquet:   0%|          | 0.00/230M [00:00<?, ?B/s]

(…)-00010-of-00011-dfef6d372d63bc4c.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28086774 [00:00<?, ? examples/s]

In [4]:
# 4. Data preprocessing
# Clean date_published
df['date_published'] = df['date_published'].astype(str).str.extract(r"((?:19|20)\d{2})", expand=False)

# Standardize ISBN13
from isbnlib import to_isbn13
def standardize_isbn(isbn):
    try:
        return to_isbn13(isbn)
    except:
        return None
df['isbn13'] = df['isbn13'].apply(standardize_isbn)


# Remove missing values and duplicates
df = df.dropna(subset=['title', 'authors', 'date_published'])
df = df.drop_duplicates(subset=['title', 'isbn13'])

In [5]:
# 5. Prepare text for tokenization
def prepare_text(row):
    return f'''<s>[INST] <<SYS>> System prompt: This is a isbn13 question-answering session. <</SYS>>
    What is the isbn13 number of the book with these details :
    Title: {row['title']}
    Long Title: {row['title_long']}
    Author(s): {row['authors']}
    Language: {row['language']}
    Published: {row['date_published']}
    ISBN: {row['isbn']} [/INST]
    The ISBN13 number is {row['isbn13']} </s>'''

# Update texts with new format
texts = df.apply(prepare_text, axis=1)

In [6]:
class BookDataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer, max_length=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __getitem__(self, idx):
        # Format the text in a more structured way
        text = self.texts[idx]

        item = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        labels = item['input_ids'].clone()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            'input_ids': item['input_ids'].squeeze(),
            'attention_mask': item['attention_mask'].squeeze(),
            'labels': labels.squeeze()
        }

    def __len__(self):
        return len(self.texts)

# Create dataset
dataset = BookDataset(texts.tolist(), tokenizer, max_length=256)

# Test the dataset
sample = dataset[0]
print("Input shape:", sample['input_ids'].shape)
print("Attention mask shape:", sample['attention_mask'].shape)
print("Labels shape:", sample['labels'].shape)
print("\nDecoded text:")
print(tokenizer.decode(sample['input_ids'][sample['attention_mask'] == 1]))

# Optional: Check how many tokens are actually being used
num_tokens = (sample['attention_mask'] == 1).sum()
print(f"\nNumber of actual tokens (non-padding): {num_tokens}")

Input shape: torch.Size([256])
Attention mask shape: torch.Size([256])
Labels shape: torch.Size([256])

Decoded text:
<s>[INST] <<SYS>> System prompt: This is a isbn13 question-answering session. <</SYS>>
    What is the isbn13 number of the book with these details :
    Title: Digital Photography All-in-One Desk Reference For Dummies
    Long Title: Digital Photography All-in-One Desk Reference For Dummies
    Author(s): ['Busch, David D.']
    Language: en
    Published: 2008
    ISBN: 0470401958 [/INST]
    The ISBN13 number is 9780470401958 </s>

Number of actual tokens (non-padding): 147


In [7]:
# Set random seeds for reproducibility
import numpy as np
import torch
np.random.seed(42)
torch.manual_seed(42)
# Calculate subset size and validation split
SUBSET_FRACTION = 1  # 1% of full dataset
VALID_FRACTION = 0.1   # 10% of subset for validation
# Convert texts to list if it's a pandas Series
texts_list = texts.tolist() if hasattr(texts, 'tolist') else texts
subset_size = int(len(texts_list) * SUBSET_FRACTION)

# Create subset indices and get subset
subset_indices = np.random.choice(len(texts_list), size=subset_size, replace=False)
texts_subset = [texts_list[i] for i in subset_indices]

# Split subset into train and validation
valid_size = int(len(texts_subset) * VALID_FRACTION)
train_size = len(texts_subset) - valid_size

train_texts = texts_subset[:-valid_size]
valid_texts = texts_subset[-valid_size:]

In [8]:
# Create train and validation datasets
train_dataset = BookDataset(train_texts, tokenizer, max_length=128)
valid_dataset = BookDataset(valid_texts, tokenizer, max_length=128)

# Print dataset sizes
print(f"Full dataset size: {len(texts_list):,}")
print(f"Subset size: {len(texts_subset):,}")
print(f"Training set size: {len(train_dataset):,}")
print(f"Validation set size: {len(valid_dataset):,}")


Full dataset size: 18,745
Subset size: 18,745
Training set size: 16,871
Validation set size: 1,874


In [9]:
# Verify samples from both splits
def inspect_dataset(dataset, name):
    sample = dataset[0]
    print(f"\n{name} Sample:")
    print("Input shape:", sample['input_ids'].shape)
    print("Decoded text:")
    print(tokenizer.decode(sample['input_ids'][sample['attention_mask'] == 1]))
    num_tokens = (sample['attention_mask'] == 1).sum()
    print(f"Number of tokens: {num_tokens}")

inspect_dataset(train_dataset, "Training")
inspect_dataset(valid_dataset, "Validation")


Training Sample:
Input shape: torch.Size([128])
Decoded text:
<s>[INST] <<SYS>> System prompt: This is a isbn13 question-answering session. <</SYS>>
    What is the isbn13 number of the book with these details :
    Title: Santuario
    Long Title: Santuario
    Author(s): ['Faulkner, William']
    Language: es
    Published: 1982
    ISBN: 847530088X [/INST]
    The ISBN13 number is 9788475300887 </s>
Number of tokens: 128

Validation Sample:
Input shape: torch.Size([128])
Decoded text:
<s>[INST] <<SYS>> System prompt: This is a isbn13 question-answering session. <</SYS>>
    What is the isbn13 number of the book with these details :
    Title: La Tierra, Planeta Vivo
    Long Title: La Tierra, Planeta Vivo
    Author(s): ['César Casquet']
    Language: es
    Published: 1985
    ISBN: 8434578530 [/INST]
    The ISBN13 number
Number of tokens: 128


In [10]:
# Configure training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,
    learning_rate=5e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    weight_decay=0.01,
    fp16=True,
    gradient_checkpointing=True,
    evaluation_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=100,
    report_to="tensorboard",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
# Function to setup LoRA model
def setup_lora_model(base_model, lora_config):
    print(f"\nPreparing model with LoRA rank {lora_config.r}")
    model = prepare_model_for_kbit_training(base_model)
    lora_model = get_peft_model(model, lora_config)
    print("\nTrainable parameters:")
    lora_model.print_trainable_parameters()
    return lora_model


In [12]:
# 2. Configure LoRA
lora_configs = [
    LoraConfig(
        r=r,  # rank
        lora_alpha=16,  # Reduced from 32 to be more conservative
        target_modules=[
            "c_attn",   # attention matrices
            "c_proj",   # projection matrices
            "c_fc",     # feed-forward components
            "c_mlp"     # MLP components
        ],
        lora_dropout=0.1,  #adjusted from 0.05 # Reduced dropout for better stability
        bias="none",
        task_type="CAUSAL_LM",
        inference_mode=False,
        fan_in_fan_out=False,  # Set to True for specific GPT-2 attention layers
        modules_to_save=None   # Optional: specify modules to fully fine-tune
    ) for r in [4, 8, 16]  # Increased rank values for better expressivity
]

In [13]:
train_dataset[0]

{'input_ids': tensor([   27,    82, 36937, 38604,    60,  9959,    50, 16309,  4211,  4482,
          6152,    25,   770,   318,   257,   318,  9374,  1485,  1808,    12,
           504,    86,  1586,  6246,    13,  1279,  3556,    50, 16309,  4211,
           198,   220,   220,   220,  1867,   318,   262,   318,  9374,  1485,
          1271,   286,   262,  1492,   351,   777,  3307,  1058,   198,   220,
           220,   220, 11851,    25, 10844,    84,  4982,   198,   220,   220,
           220,  5882, 11851,    25, 10844,    84,  4982,   198,   220,   220,
           220,  6434,     7,    82,  2599, 37250,    37,  2518,    74,  1008,
            11,  3977, 20520,   198,   220,   220,   220, 15417,    25,  1658,
           198,   220,   220,   220, 26372,    25, 14489,   198,   220,   220,
           220, 32429,    25,  9508,  2425,  6200,  3459,    55, 46581, 38604,
            60,   198,   220,   220,   220,   383, 32429,  1485,  1271,   318,
         10111,  3459, 32576,  6200, 46

In [15]:
from transformers import EarlyStoppingCallback
# Training function
def train_with_config(config_idx, lora_config, model, train_dataset, valid_dataset):
    print(f"\n{'='*50}")
    print(f"Starting training for LoRA rank {lora_config.r} (Configuration {config_idx + 1}/3)")
    print(f"{'='*50}")

    # Setup model with LoRA
    lora_model = setup_lora_model(model, lora_config)

    # Initialize trainer
    trainer = Trainer(
        model=lora_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=DataCollatorForLanguageModeling(
            tokenizer=tokenizer,
            mlm=False
        ),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Added early stopping
    )

    # Calculate and print training statistics
    total_steps = int(len(train_dataset) / (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs)
    print(f"\nTraining Statistics:")
    print(f"Total training steps: {total_steps}")
    print(f"Evaluation every {training_args.eval_steps} steps")
    print(f"Save checkpoint every {training_args.save_steps} steps")

    # Train and evaluate
    print("\nStarting training...")
    train_result = trainer.train()
    print("\nRunning final evaluation...")
    eval_result = trainer.evaluate()

    # Save final model
    print(f"\nSaving final model for rank {lora_config.r}...")
    trainer.save_model(f"./final_model_rank_{lora_config.r}")
    trainer.model.push_to_hub(f"hitmanonholiday/gpt2-large-{lora_config.r}")
    model.push_to_hub(f"hitmanonholiday/gpt2-large-{lora_config.r}")
    tokenizer.push_to_hub(f"hitmanonholiday/gpt2-large-{lora_config.r}")

    # Print results
    print(f"\nResults for LoRA rank {lora_config.r}:")
    print(f"Final training loss: {train_result.training_loss:.4f}")
    print(f"Final validation loss: {eval_result['eval_loss']:.4f}")
    print(f"Training time: {train_result.metrics['train_runtime']:.2f} seconds")

    return train_result, eval_result, trainer

# Run training for all LoRA configurations
results = []
for idx, config in enumerate(lora_configs):
    try:
        train_result, eval_result,trainer = train_with_config(
            idx, config, model, train_dataset, valid_dataset
        )
        results.append({
            'rank': config.r,
            'train_result': train_result,
            'eval_result': eval_result
        })
    except Exception as e:
        print(f"\nError training rank {config.r}: {str(e)}")
        continue

# Print final comparison
print("\n" + "="*50)
print("Training Complete! Final Results:")
print("="*50)
for result in results:
    print(f"\nRank {result['rank']}:")
    print(f"  Final Training Loss: {result['train_result'].training_loss:.4f}")
    print(f"  Final Validation Loss: {result['eval_result']['eval_loss']:.4f}")
    print(f"  Training Time: {result['train_result'].metrics['train_runtime']:.2f} seconds")

# Save results to file
import json
from datetime import datetime

results_summary = {
    'timestamp': datetime.now().strftime("%Y-%m-%d_%H-%M-%S"),
    'results': [{
        'rank': r['rank'],
        'train_loss': float(r['train_result'].training_loss),
        'eval_loss': float(r['eval_result']['eval_loss']),
        'runtime': float(r['train_result'].metrics['train_runtime'])
    } for r in results]
}

with open('training_results.json', 'w') as f:
    json.dump(results_summary, f, indent=2)

print("\nResults saved to training_results.json")


Starting training for LoRA rank 4 (Configuration 1/3)

Preparing model with LoRA rank 4

Trainable parameters:
trainable params: 2,949,120 || all params: 776,979,200 || trainable%: 0.3796

Training Statistics:
Total training steps: 2636
Evaluation every 500 steps
Save checkpoint every 500 steps

Starting training...


Step,Training Loss,Validation Loss
500,0.866600,0.831696
1000,0.841800,0.814529
1500,0.809600,0.808463
2000,0.765200,0.807676
2500,0.757800,0.808013



Running final evaluation...



Saving final model for rank 4...


adapter_model.safetensors:   0%|          | 0.00/11.8M [00:00<?, ?B/s]

...


Saving final model for rank 16...


adapter_model.safetensors:   0%|          | 0.00/47.2M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.14G [00:00<?, ?B/s]


Results for LoRA rank 16:
Final training loss: 0.8565
Final validation loss: 0.8062
Training time: 1562.44 seconds

Training Complete! Final Results:

Rank 4:
  Final Training Loss: 0.8565
  Final Validation Loss: 0.8077
  Training Time: 1549.91 seconds

Rank 8:
  Final Training Loss: 0.8563
  Final Validation Loss: 0.8064
  Training Time: 1541.80 seconds

Rank 16:
  Final Training Loss: 0.8565
  Final Validation Loss: 0.8062
  Training Time: 1562.44 seconds

Results saved to training_results.json
